In [2]:
# buy stocks cheaper than "intrinsic" value

In [3]:
import numpy as np
import pandas as pd
import requests
from scipy import stats

In [4]:
stocks = pd.read_csv('../sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

In [5]:
batches = [stocks[:86]['Ticker'], stocks[86: 192]['Ticker'], stocks[192:277]['Ticker'], stocks[277: 359]['Ticker'], stocks[359: 444]['Ticker'], stocks[444:]['Ticker']]

In [14]:
cols = ['Ticker', 'Price', 'PE Ratio', '#Shares to buy']
dictionary = {col: list() for col in cols}

In [15]:
def batch_calls(tickers):
    tickers = ','.join(list(tickers))
    batch_url = f"https://sandbox.iexapis.com/stable/stock//market/batch?symbols={tickers}&types=quote&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batch_url)
    if data.status_code != 200:
        return None
    return data.json()

In [16]:
for batch in batches:
    data = batch_calls(batch)
    
    for ticker in batch:
        if ticker in data:
            dictionary['Ticker'].append(ticker)
            dictionary['Price'].append(data[ticker]['quote']['latestPrice'])
            dictionary['PE Ratio'].append(data[ticker]['quote']['peRatio'])
            dictionary['#Shares to buy'].append('N/A')
        

In [17]:
df = pd.DataFrame(dictionary)

In [18]:
df

,Ticker,Price,PE Ratio,#Shares to buy
0,A,118.70,28.31,N/A
1,AAL,13.20,-3.75,N/A
2,AAP,180.40,19.81,N/A
3,AAPL,139.92,22.58,N/A
4,ABBV,153.99,21.87,N/A
...,...,...,...,...
490,YUM,114.43,21.07,N/A
491,ZBH,106.29,107.76,N/A
492,ZBRA,299.60,19.70,N/A
493,ZION,52.98,5.55,N/A


In [21]:
df.sort_values('PE Ratio', inplace=True, ignore_index=True)

Lower the PE Ratio is "better"

In [22]:
df = df[df['PE Ratio'] > 0]
df = df[:50]

In [24]:
df

,Ticker,Price,PE Ratio,#Shares to buy
22,NRG,36.91,2.31,N/A
23,EBAY,43.36,2.38,N/A
24,AMZN,113.58,2.66,N/A
25,SYF,30.06,4.00,N/A
26,F,11.73,4.00,N/A
27,NUE,109.71,4.01,N/A
28,COF,107.82,4.32,N/A
29,PVH,62.10,4.42,N/A
30,AIG,53.65,4.60,N/A
31,DHI,66.84,4.67,N/A


In [25]:
portfolio_size = 10e7
cap_alloc = portfolio_size//50

df['#Shares to buy'] = cap_alloc//df['Price']

In [26]:
df

,Ticker,Price,PE Ratio,#Shares to buy
22,NRG,36.91,2.31,54185.0
23,EBAY,43.36,2.38,46125.0
24,AMZN,113.58,2.66,17608.0
25,SYF,30.06,4.00,66533.0
26,F,11.73,4.00,170502.0
27,NUE,109.71,4.01,18229.0
28,COF,107.82,4.32,18549.0
29,PVH,62.10,4.42,32206.0
30,AIG,53.65,4.60,37278.0
31,DHI,66.84,4.67,29922.0


# Improved Model

In [27]:
def batch_calls(tickers):
    tickers = ','.join(list(tickers))
    batch_url = f"https://sandbox.iexapis.com/stable/stock//market/batch?symbols={tickers}&types=advanced-stats,quote&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batch_url)
    if data.status_code != 200:
        return None
    return data.json()

In [28]:
data = batch_calls(batches[0])

In [30]:
batches = [stocks[:86]['Ticker'], stocks[86: 192]['Ticker'], stocks[192:277]['Ticker'], stocks[277: 359]['Ticker'], stocks[359: 444]['Ticker'], stocks[444:]['Ticker']]

In [50]:
rv_cols = rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

dictionary = {col: [] for col in rv_cols}

In [51]:
for batch in batches:
    data = batch_calls(batch)
    
    if not data:
        continue
        
    for ticker in batch:
        if ticker in data:
            dictionary['Ticker'].append(ticker)
            dictionary['Price'].append(data[ticker]['quote']['latestPrice'])
            dictionary['Number of Shares to Buy'].append('N/A')
            dictionary['Price-to-Earnings Ratio'].append(data[ticker]['quote']['peRatio'])
            dictionary['PE Percentile'].append('N/A')
            dictionary['Price-to-Book Ratio'].append(data[ticker]['advanced-stats']['priceToBook'])
            dictionary['PB Percentile'].append('N/A')
            dictionary['Price-to-Sales Ratio'].append(data[ticker]['advanced-stats']['priceToSales'])
            dictionary['PS Percentile'].append('N/A')
            enterprise = data[ticker]['advanced-stats']['enterpriseValue']
            ebitda = data[ticker]['advanced-stats']['EBITDA']
            
            if enterprise and ebitda:
                dictionary['EV/EBITDA'].append(enterprise/ebitda)
            else:
                dictionary['EV/EBITDA'].append('N/A')                
            dictionary['EV/EBITDA Percentile'].append('N/A')
            gp = data[ticker]['advanced-stats']['grossProfit']
            if not gp or not enterprise:
                dictionary['EV/GP'].append('N/A')
            else:
                dictionary['EV/GP'].append(enterprise/gp)
            dictionary['EV/GP Percentile'].append('N/A')
            dictionary['RV Score'].append('N/A')

In [52]:
df = pd.DataFrame(dictionary)

In [53]:
df

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,114.70,N/A,27.69,N/A,6.7600,N/A,5.3800,N/A,20.519517,N/A,10.603254,N/A,N/A
1,AAL,13.70,N/A,-3.74,N/A,-0.9892,N/A,0.2443,N/A,-20.879419,N/A,0.95062,N/A,N/A
2,AAP,180.61,N/A,19.69,N/A,3.6300,N/A,0.9448,N/A,11.084304,N/A,2.354238,N/A,N/A
3,AAPL,140.45,N/A,22.03,N/A,33.5700,N/A,5.8000,N/A,17.430749,N/A,13.730136,N/A,N/A
4,ABBV,148.39,N/A,21.44,N/A,16.1300,N/A,4.6800,N/A,10.883678,N/A,7.835986,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,YUM,113.37,N/A,20.82,N/A,-3.8300,N/A,4.8200,N/A,18.919327,N/A,8.752049,N/A,N/A
491,ZBH,106.70,N/A,107.59,N/A,1.8000,N/A,2.8000,N/A,16.536893,N/A,5.047216,N/A,N/A
492,ZBRA,300.77,N/A,20.30,N/A,5.5200,N/A,2.7400,N/A,15.18859,N/A,6.128758,N/A,N/A
493,ZION,53.46,N/A,5.70,N/A,1.3700,N/A,2.3200,N/A,4.338415,N/A,2.217337,N/A,N/A


In [54]:
df[df.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
28,ALXN,183.400,N/A,62.02,N/A,NaN,N/A,NaN,N/A,N/A,N/A,N/A,N/A,N/A
40,AON,257.150,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A,N/A,N/A,N/A,N/A
71,BRK.B,277.810,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A,N/A,N/A,N/A,N/A
88,CERN,98.730,N/A,50.00,N/A,NaN,N/A,NaN,N/A,N/A,N/A,N/A,N/A,N/A
118,CTL,11.000,N/A,10.13,N/A,NaN,N/A,NaN,N/A,N/A,N/A,N/A,N/A,N/A
135,DISCK,25.240,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A,N/A,N/A,N/A,N/A
164,ETFC,49.780,N/A,14.46,N/A,NaN,N/A,NaN,N/A,N/A,N/A,N/A,N/A,N/A
186,FRT,95.440,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A,N/A,N/A,N/A,N/A
197,GOOG,2251.170,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A,N/A,N/A,N/A,N/A
317,MXIM,106.510,N/A,35.00,N/A,NaN,N/A,NaN,N/A,N/A,N/A,N/A,N/A,N/A


In [73]:
for col in df.select_dtypes(include='number'):
    df[col].fillna(df[col].mean(), inplace= True)

In [76]:
df[df.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,114.70,N/A,27.69,70.445344,6.7600,80.566802,5.3800,81.983806,20.519517,NaN,10.603254,N/A,N/A
1,AAL,13.70,N/A,-3.74,3.238866,-0.9892,5.870445,0.2443,0.809717,-20.879419,NaN,0.95062,N/A,N/A
2,AAP,180.61,N/A,19.69,49.190283,3.6300,56.882591,0.9448,14.777328,11.084304,NaN,2.354238,N/A,N/A
3,AAPL,140.45,N/A,22.03,54.655870,33.5700,96.963563,5.8000,82.793522,17.430749,NaN,13.730136,N/A,N/A
4,ABBV,148.39,N/A,21.44,53.238866,16.1300,92.914980,4.6800,76.113360,10.883678,NaN,7.835986,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,YUM,113.37,N/A,20.82,51.619433,-3.8300,5.465587,4.8200,77.125506,18.919327,NaN,8.752049,N/A,N/A
491,ZBH,106.70,N/A,107.59,98.380567,1.8000,26.923077,2.8000,52.226721,16.536893,NaN,5.047216,N/A,N/A
492,ZBRA,300.77,N/A,20.30,50.607287,5.5200,74.291498,2.7400,50.404858,15.18859,NaN,6.128758,N/A,N/A
493,ZION,53.46,N/A,5.70,8.906883,1.3700,18.218623,2.3200,43.117409,4.338415,NaN,2.217337,N/A,N/A


In [67]:
sz = df['Price-to-Earnings Ratio'].size -1
df['PE Percentile'] = df['Price-to-Earnings Ratio'].rank(method='max').apply(lambda x: 100 * (x -1)/sz)

In [68]:
sz = df['Price-to-Book Ratio'].size -1
df['PB Percentile'] = df['Price-to-Book Ratio'].rank(method='max').apply(lambda x: 100 * (x -1)/sz)

In [69]:
sz = df['Price-to-Sales Ratio'].size -1
df['PS Percentile'] = df['Price-to-Sales Ratio'].rank(method='max').apply(lambda x: 100 * (x -1)/sz)

In [80]:
sz = df['EV/EBITDA'].size -1
df['EV/EBITDA Percentile'] = df['EV/EBITDA'].rank(method='max').apply(lambda x: 100 * (x -1)/sz)

/var/folders/cg/pm4dk58d6ndgwns95k12lkfw0000gn/T/ipykernel_14346/2431338324.py:2: FutureWarning: Dropping of nuisance columns in Series.rank is deprecated; in a future version this will raise TypeError. Select only valid columns before calling rank.
  df['EV/EBITDA Percentile'] = df['EV/EBITDA'].rank(method='max').apply(lambda x: 100 * (x -1)/sz)


In [79]:
df['EV/EBITDA']

0      20.519517
1     -20.879419
2      11.084304
3      17.430749
4      10.883678
         ...    
490    18.919327
491    16.536893
492     15.18859
493     4.338415
494    26.279588
Name: EV/EBITDA, Length: 495, dtype: object